In [ ]:
import pandas as pd
import numpy as np

In [ ]:
steam = pd.read_csv('steam.csv')
steam_description_data = pd.read_csv('steam_description_data.csv')
steam_media_data = pd.read_csv('steam_media_data.csv')
steam_requirements_data = pd.read_csv('steam_requirements_data.csv')
steam_support_info = pd.read_csv('steam_support_info.csv')

## Game

In [ ]:
steam_new = steam[['appid','name','publisher','platforms','positive_ratings',
                   'negative_ratings','price','release_date']]

In [ ]:
steam_new['rating'] = steam_new['positive_ratings']/(steam_new['positive_ratings']+steam_new['negative_ratings'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
steam_new = steam_new.drop(columns=['positive_ratings','negative_ratings'])

In [ ]:
headers_movies = steam_media_data[['steam_appid','header_image','movies']]

In [ ]:
game_temp = steam_new.merge(headers_movies,left_on='appid',right_on='steam_appid').drop(columns=['steam_appid'])

In [ ]:
game_df = game_temp.merge(steam_description_data,left_on='appid',right_on='steam_appid').\
drop(columns=['detailed_description','about_the_game'])

In [ ]:
game_df = game_df.rename(columns={'appid':'game_id',\
                                  'header_image':'header','publisher':'company'})\
                                  .drop(columns=['steam_appid'])                      

In [ ]:
game_df['rating'] = (game_df['rating']*10).round(decimals=2)

In [ ]:
empty_indx = list(game_df[game_df['company']==" "].index)

In [ ]:
game_df = game_df.drop(labels=empty_indx,axis=0)

In [ ]:
game_df

,game_id,name,company,platforms,price,release_date,rating,header,movies,short_description
0,10,Counter-Strike,Valve,windows;mac;linux,7.19,2000-11-01,9.74,https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,Valve,windows;mac;linux,3.99,1999-04-01,8.40,https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,One of the most popular online action games of...
2,30,Day of Defeat,Valve,windows;mac;linux,3.99,2003-05-01,8.96,https://steamcdn-a.akamaihd.net/steam/apps/30/...,NaN,Enlist in an intense brand of Axis vs. Allied ...
3,40,Deathmatch Classic,Valve,windows;mac;linux,3.99,2001-06-01,8.27,https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,Enjoy fast-paced multiplayer gaming with Death...
4,50,Half-Life: Opposing Force,Valve,windows;mac;linux,3.99,1999-11-01,9.48,https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN,Return to the Black Mesa Research Facility as ...
...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,SHEN JIAWEI,windows,2.09,2019-04-24,10.00,https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 256747452, 'name': 'Video', 'thumbnail...",The Room of Pandora is a third-person interact...
27071,1065570,Cyber Gun,BekkerDev Studio,windows,1.69,2019-04-23,8.89,https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 256747487, 'name': 'Game Trailer', 'th...",Cyber Gun is a hardcore first-person shooter w...
27072,1065650,Super Star Blast,EntwicklerX,windows,3.99,2019-04-24,0.00,https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 256747530, 'name': 'Gameplay Trailer',...",Super Star Blast is a space based game with ch...
27073,1066700,New Yankee 7: Deer Hunters,Alawar Entertainment,windows;mac,5.19,2019-04-17,10.00,https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 256747638, 'name': 'Main Trailer', 'th...",Pursue a snow-white deer through an enchanted ...


In [ ]:
game_df.to_csv('Game.csv',index=False)

##Game_tag

In [ ]:
dic = {}
for i,j in zip(steam['appid'], steam['steamspy_tags'].apply(lambda x: x.split(';'))):
  dic[i] = j

# make sure each game_id has same amount of tags
for x, y in dic.items():
  while True:
    if len(y)!=3:
      dic[x].append('Null')
    else:
      break

In [ ]:
indx_lst = []
for i,j in dic.items():
  for _ in range(3):
    indx_lst.append(i)

In [ ]:
tags_lst = []
for i,j in dic.items():
  for v in j:
    tags_lst.append(v)

In [ ]:
game_tag_df = pd.DataFrame({'tag_name':tags_lst},index=indx_lst)
game_tag_df = game_tag_df[game_tag_df['tag_name']!='Null'].reset_index()\
.rename(columns={'index':'game_id'})

In [ ]:
game_tag_df.to_csv('Game_Tag.csv',index=False)

##Tag

In [ ]:
tags = []
for i in [x.split(';') for x in list(steam['steamspy_tags'].values)]:
  for j in i:
    if j not in tags:
      tags.append(j)

In [ ]:
tag_df = pd.DataFrame(np.array(tags).reshape(len(tags),1), columns = ['tag_name'])

In [ ]:
tag_df.to_csv('Tag.csv',index=False)

## System

In [ ]:
systems = []
for i in [x.split(';') for x in list(steam['platforms'].values)]:
  for j in i:
    if j not in systems:
      systems.append(j)


system_df = pd.DataFrame(np.array(systems).reshape(len(systems),1), columns = ['system_name'])

In [ ]:
system_df.to_csv('System.csv',index=False)

##Company

In [ ]:
game_df.groupby('company').agg({'rating':['count','mean']}).reset_index()

company rating           
                             count       mean
0          Milestone S.r.l.      1   6.670000
1              AFBIK Studio      3   8.023333
2            AYE Technology      1   2.500000
3              Anea_Duo_Dev      1   5.560000
4            Asterion Games      1   8.020000
...                     ...    ...        ...
14348                  飞鹰互娱      1   9.090000
14349  香港商河洛互動娛樂股份有限公司;方块游戏      1   5.890000
14350                  魂动天下      1  10.000000
14351                  魔力乐章      1   8.570000
14352                   수프림      1  10.000000

[14353 rows x 3 columns]

In [ ]:
game_df.groupby('company').agg({'rating':['count','mean']}).reset_index()['rating']

,count,mean
0,1,6.670000
1,3,8.023333
2,1,2.500000
3,1,5.560000
4,1,8.020000
...,...,...
14348,1,9.090000
14349,1,5.890000
14350,1,10.000000
14351,1,8.570000


In [ ]:
company = \
game_df.groupby('company').agg({'rating':['count','mean']}).reset_index()[['company']]

In [ ]:
count = game_df.groupby('company').agg({'rating':['count','mean']}).reset_index()['rating']['count']
mean = game_df.groupby('company').agg({'rating':['count','mean']}).reset_index()['rating']['mean']

In [ ]:
company['count'] = count
company['mean'] = mean

In [ ]:
company = company.rename(columns={'count':'num_game','mean':'avg_rating'})

In [ ]:
company = company.rename(columns={'company':'name'})

In [ ]:
company['avg_rating'] = company['avg_rating'].round(decimals=2)

In [ ]:
company.to_csv('Company.csv',index=False)